In [4]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson import json_util
from time import time
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import requests
from urllib.parse import unquote
import unidecode
import re
import requests
from bs4 import BeautifulSoup
import json
import pickle

In [5]:
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return json.JSONEncoder.default(self, o)

In [6]:
def split_names(s,exceptions=['GIL','LEW','LIZ','PAZ','REY','RIO','ROA','RUA','SUS','ZEA']):
    """
    Extract the parts of the full name `s` in the format ([] → optional):
    
    [SMALL_CONECTORS] FIRST_LAST_NAME [SMALL_CONECTORS] [SECOND_LAST_NAME] NAMES
    
    * If len(s) == 2 → Foreign name assumed with single last name on it
    * If len(s) == 3 → Colombian name assumed two last mames and one first name
    
    Add short last names to `exceptions` list if necessary
    
    Works with:
    ----
        s='LA ROTTA FORERO DANIEL ANDRES'
        s='MONTES RAMIREZ MARIA DEL CONSUELO'
        s='CALLEJAS POSADA RICARDO DE LA MERCED'
        s='DE LA CUESTA BENJUMEA MARIA DEL CARMEN'
        s='JARAMILLO OCAMPO NICOLAS CARLOS MARTI'
        s='RESTREPO QUINTERO DIEGO ALEJANDRO'
        s='RESTREPO ZEA JAIRO HUMBERTO'
        s='JIMENEZ DEL RIO MARLEN'        
        s='RESTREPO FERNÁNDEZ SARA' # Colombian: two LAST_NAMES NAME
        s='NARDI ENRICO' # Foreing
    Fails:
    ----
        s='RANGEL MARTINEZ VILLAL ANDRES MAURICIO' # more than 2 last names
        s='ROMANO ANTONIO ENEA' # Foreing → LAST_NAME NAMES
    """
    s=s.title()
    exceptions=[e.title() for e in exceptions]
    sl=re.sub('(\s\w{1,3})\s',r'\1-',s,re.UNICODE)
    sl=re.sub('(\s\w{1,3}\-\w{1,3})\s',r'\1-',sl,re.UNICODE)
    sl=re.sub('^(\w{1,3})\s',r'\1-' ,sl,re.UNICODE)
    #Clean exceptions
    #Extract short names list
    lst=[s for s in re.split( '(\w{1,3})\-',sl ) if len(s)>=1 and len(s)<=3 ]
    #intersection with exceptions list
    exc=[value for value in exceptions if value in lst]
    if exc:
        for e in exc:
            sl=sl.replace('{}-'.format(e),'{} '.format(e))
            
    #if sl.find('-')>-1:
    #print(sl)
    sll=[s.replace('-',' ') for s in sl.split()]
    if len(s.split())==2:
        sll=[s.split()[0]]+['']+[s.split()[1]]
    #
    d={'NOMBRE COMPLETO' : ' '.join(sll[2:]+sll[:2]),
     'PRIMER APELLIDO' : sll[0], 
     'SEGUNDO APELLIDO': sll[1], 
     'NOMBRES'         :' '.join(sll[2:]), 
     'INICIALES'       :' '.join( [i[0]+'.' for i in ' '.join(sll[2:]).split() ] )
    }
    return d

In [7]:
# Mise à jour des noms dans différentes langues via wikipedia
def get_wikipedia_names(url="",name="",lang="en",verbose=0):
    '''
    Find the different possible names of a wikipedia entity.
    Right now it is only tested on organizations gotten from ror db
    
    Parameters
    ----------
    url : str
        The wikipedia url if is available
    name : str
        The name of keywords to do the search over wikipedia api
    lang : str
        The iso-639 lang code to fix the language endpooint of the search language
        
    Returns
    -------
    data : dict
        The response of the wikipedia requests with the langlinks of the prop params
        
    '''
    if url:
        subject = unquote(url.split("/")[-1].replace("_"," "))
    elif name:
        subject = name
    else:
        return {"response":[],"names":[]}
    
    base = 'https://'+lang+'.wikipedia.org/w/api.php'
    #searching entire wikipedia
    if verbose>5:
        print("Searching ",subject)
    params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'srsearch':subject
        }
 
    data = requests.get(base, params=params).json()
    #print(data)
    entry=""
    pageid=""
    if not "query" in data.keys():
        return None
    for reg in data["query"]["search"]: #searching among the results and checking twice with fuzzywuzzy
        score=fuzz.ratio(reg["title"].lower(),subject.lower())
        if score>90:
            entry=reg
            pageid=int(reg["pageid"])
        elif score>50:
            score=fuzz.partial_ratio(reg["title"].lower(),subject.lower())
            if score>95:
                entry=reg
                pageid=int(reg["pageid"])
            elif score>80:
                score=fuzz.token_set_ratio(reg["title"].lower(),subject.lower())
                if score>98:
                    entry=reg
                    pageid=int(reg["pageid"])
        if entry!="":
            break

    if pageid!="": #if the page id is available
        #retrieving the actual page's langlinks 
        params = {
                'action': 'query',
                'format': 'json',
                'pageids': pageid,
                'prop': 'langlinks',
                'lllimit':500,
                #'exintro': True,
                #'explaintext': True,
            }

        response = requests.get(base, params=params)
        data = response.json()
        return data
    else:
        return None

In [8]:
client=MongoClient()
colombia=client["colombia_udea"]
openalex=client["openalex"]
ror=client["ror"]

In [9]:
def empty_affiliations():
    entry = {
        "updated":[],
        "names":[],
        "aliases":[],
        "abbreviations":[],
        "types":[],
        "year_established":-1,
        "status":[],
        "relations":[],
        "addresses":[],
        "external_urls":[],
        "external_ids":[],
        "subjects":[],
        "ranking":[],
        "description":[]
    }
    return entry

### Inserting ROR

In [23]:
%%timeit -n 1 -r 1
already_indb=0
inserted=0
updated=0
for inst in ror["stage"].find():
    #print(inst)
    found_entry=colombia["affiliations"].find_one({"external_ids.id":inst["id"]})
    if found_entry:
        #print("Found {} in the database with id {}".format(found_entry["names"],found_entry["_id"]))
        already_indb+=1
        continue
    entry=empty_affiliations()
    
    entry["updated"].append({"time":int(time()),"source":"ror"})
    entry["names"].append({"name":inst["name"],"lang":"en"})
    entry["aliases"].extend(inst["aliases"])
    entry["abbreviations"].extend(inst["acronyms"])
    entry["year_established"]=int(inst["established"]) if inst["established"] else -1
    entry["status"]=[inst["status"]]
    
    for typ in inst["types"]:
        entry["types"].append({"source":"ror","type":typ})

    for add in inst["addresses"]:
        add_entry={
            "lat":add["lat"],
            "lng":add["lng"],
            "postcode":add["postcode"] if add["postcode"] else "",
            "state":add["state"],
            "city":add["city"],
            "country":"",
            "country_code":"",
        }
        entry["addresses"].append(add_entry)
    
    #external_urls
    if inst["links"]:
        for link in inst["links"]:
            url_entry={"source":"site","url":inst["links"][0]}
            if not url_entry in entry["external_urls"]:
                entry["external_urls"].append(url_entry)
    #country
    entry["addresses"][0]["country"]=inst["country"]["country_name"]
    entry["addresses"][0]["country_code"]=inst["country"]["country_code"]
    #external_ids
    if inst["external_ids"]:
        for key,ext in inst["external_ids"].items():
            if isinstance(ext["all"],list):
                alll=ext["all"][0] if len(ext["all"])>0 else ext["all"]
                ext_entry={"source":key.lower(),"id":alll}
                if not ext_entry in entry["external_ids"]:
                    entry["external_ids"].append(ext_entry)
    entry["external_ids"].append({"source":"ror","id":inst["id"]})
    if inst["wikipedia_url"]:
        entry["external_urls"].append({"source":"wikipedia","url":inst["wikipedia_url"]}) 
    colombia["affiliations"].insert_one(entry)
    inserted+=1
    
print(inserted,already_indb,updated)

102993 0 0
45.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [47]:
#fixing status ONLY IF THEY ARE NOT A LIST
#for inst in colombia["affiliations"].find({"status":{"$ne":[]}},no_cursor_timeout=True):
#    status=inst["status"]
#    colombia["affiliations"].update_one({"_id":inst["_id"]},{"$set":{"status":[{"source":"ror","status":status}]}})

/usr/local/lib/python3.8/dist-packages/pymongo/collection.py:1643: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


In [24]:
wikipedialess=[]
wrong=[]
successful=0

In [ ]:
with client.start_session() as session:
    old=dt.datetime.now()
    for inst in colombia["affiliations"].find(no_cursor_timeout=True):
        if len(inst["names"])>1 or inst["_id"] in wrong or inst["_id"] in wikipedialess:
            continue
        wikipedia_url=""
        wikipedia_name=""
        for ext in inst["external_urls"]:
            if ext["source"]=="wikipedia":
                wikipedia_url=ext["url"]
                break
        if not wikipedia_url:
            #print("Could not find wikipedia url for {} with id {}".format(inst["names"][0]["name"],inst["_id"]))
            wikipedialess.append(inst["_id"])
            wikipedia_name=inst["names"][0]["name"]
        if wikipedia_url:
            if not "wiki" in wikipedia_url:
                continue
            #print("Processing ",wikipedia_url)
        elif wikipedia_name:
            pass
            #print("Processing ",wikipedia_name)
        else:
            print("No information could be used for wikipedia API query in ",inst["_id"])
            continue
        result={}
        res=[]
        if wikipedia_url:
            res=get_wikipedia_names(url=wikipedia_url)
        elif wikipedia_name:
            res=get_wikipedia_names(name=wikipedia_name)
        try:
            k=list(res["query"]["pages"].keys())[0]
            result={"response":res,"names":res["query"]["pages"][k]["langlinks"]}

        except Exception as e:
            if wikipedia_url:
                print("Something went wrong processing ",wikipedia_url)
            elif wikipedia_name:
                print("Something went wrong processing ",wikipedia_name)
            result={"response":res,"names":[]}
            #print(e)
            #print(res)
            #print("Trying spanish api")
            if wikipedia_url:
                res=get_wikipedia_names(url=wikipedia_url,lang="es")
            elif wikipedia_name:
                res=get_wikipedia_names(name=wikipedia_name,lang="es")
            try:
                k=list(res["query"]["pages"].keys())[0]
                result={"response":res,"names":res["query"]["pages"][k]["langlinks"]}

            except Exception as e:
                if wikipedia_url:
                    print("Something went wrong processing ",wikipedia_url)
                elif wikipedia_name:
                    print("Something went wrong processing ",wikipedia_name)
                result={"response":res,"names":[]}
                print(e)
                print(res)
                print("-------------------------")

        #print(result)
        if not result["names"]:
            #print("Could not get names from \n\t",res)
            wrong.append(inst["_id"])
            continue
        else:
            successful+=1
            #print(result["names"])
            names=inst["names"]
            for nam in result["names"]:
                if nam["lang"]!="en":
                    names.append({"name":nam["*"],"lang":nam["lang"]})
            #print(names)
            colombia["affiliations"].update_one({"_id":inst["_id"]},{"$set":{"names":names}})
            #print(colombia["affiliations"].count_documents({}),len(wrong),len(wikipedialess),successful)
        delta=dt.datetime.now()-old
        if delta.seconds>240:
            print(colombia["affiliations"].count_documents({}),len(wrong),len(wikipedialess),successful)
            client.admin.command('refreshSessions', [session.session_id], session=session)
            old=dt.datetime.now()
print(colombia["affiliations"].count_documents({}),len(wrong),len(wikipedialess),successful)

In [175]:
%%timeit -n 1 -r 1
updated=0
#Pour mettre à jour l'entrée des relations, nous devons itérer à nouveau dans la collection
for inst in ror["stage"].find({"relationships":{"$ne":[]}}):
    rel_entry=[]
    updatable_entry=colombia["affiliations"].find_one({"external_ids.id":inst["id"]})
    if not updatable_entry:
        print("Could not find institution with id ",inst["id"])
        continue
    for rel in inst["relationships"]:
        db_entry=colombia["affiliations"].find_one({"external_ids.id":rel["id"]})
        if db_entry:
            re_sub={"id":db_entry["_id"],"name":db_entry["names"],"types":db_entry["types"]}
            if not re_sub in rel_entry:
                rel_entry.append(re_sub)
        else:
            print("Could not find relation with the ror id ",rel["id"])
    colombia["affiliations"].update_one({"_id":updatable_entry["_id"]},{"$set":{"relations":rel_entry}})
    updated+=1
int(updated)
#print(inserted,already_indb,updated)

24.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
udea_reg=colombia["affiliations"].find_one({"names.name":"University of Antioquia"})
udea_reg

#### Creating indexes

In [ ]:
colombia["affiliations"].create_index({"names.name":"text"})

### Inserting Faculties and departments from "Talento Humano UDEA"

In [11]:
th = pd.read_excel("/current/data/colombia/udea/Base de Datos profesores 2019_con_clasificación de Colciencias.xlsx",
                   dtype={"cedula":str,"codfac":str,"ccosto":str,"fecha_nac":str,"fecha_vin":str})
th["cedula"]=[cc.split(".")[0] for cc in th["cedula"]]

In [12]:
th["Nombre fac"].unique(),len(th["Nombre fac"].unique())

(array(['Artes', 'Exactas', 'Sociales', 'Odontología', 'Derecho',
        'Est. Políticos', 'Comunicaciones', 'Educación', 'Idiomas',
        'Filosofía', 'Económicas', 'Ingeniería', 'Medicina',
        'Farmacéuticas', 'Microbiología', 'Salud Pública', 'Agrarias',
        'Bibliotecología', 'Enfermería', 'Educación Física', 'Nutrición ',
        'Corp Ambiental', 'Est. Regionales', 'Regionalización',
        'Vic Docencia'], dtype=object),
 25)

In [13]:
len(th)

1983

In [176]:
facs_inserted={}
deps_inserted={}

cedula_fac={}
cedula_dep={}
fac_dep_pair=[]


In [177]:
for idx, reg in th.iterrows():
    reg["Nombre fac"]=reg["Nombre fac"].strip()
    if reg["Nombre fac"]=='Vic Docencia':
        reg["Nombre fac"]="Vicerrectoría de Docencia"
    if reg["Nombre fac"]=="Exactas":
        reg["Nombre fac"]="Facultad de Ciencias Exactas y Naturales"
    if reg["Nombre fac"]=="Sociales":
        reg["Nombre fac"]="Facultad de Ciencias Sociales y Humanas"
    if reg["Nombre fac"]=="Derecho":
        reg["Nombre fac"]="Facultad de Derecho y Ciencias Políticas"
    if reg["Nombre fac"]=="Agrarias":
        reg["Nombre fac"]="Facultad de Ciencias Agrarias"
    if reg["Nombre fac"]=="Est. Políticos":
        reg["Nombre fac"]="Institutio de Estudios Políticos"
    if reg["Nombre fac"]=="Artes":
        reg["Nombre fac"]="Facultad de Artes"
    if reg["Nombre fac"]=="Odontología":
        reg["Nombre fac"]="Facultad de Odontología"
    if reg["Nombre fac"]=="Comunicaciones":
        reg["Nombre fac"]="Facultad de Comunicaciones y Filología"
    if reg["Nombre fac"]=="Educación":
        reg["Nombre fac"]="Facultad de Educación"
    if reg["Nombre fac"]=="Idiomas":
        reg["Nombre fac"]="Escuela de Idiomas"
    if reg["Nombre fac"]=="Filosofía":
        reg["Nombre fac"]="Instituto de Filosofía"
    if reg["Nombre fac"]=="Económicas":
        reg["Nombre fac"]="Facultad de Ciencias Económicas"
    if reg["Nombre fac"]=="Ingeniería":
        reg["Nombre fac"]="Facultad de Ingeniería"
    if reg["Nombre fac"]=="Medicina":
        reg["Nombre fac"]="Facultad de Medicina"
    if reg["Nombre fac"]=="Farmacéuticas":
        reg["Nombre fac"]="Facultad de Ciencias Farmacéuticas y Alimentarias"
    if reg["Nombre fac"]=="Microbiología":
        reg["Nombre fac"]="Escuela de Microbiología"
    if reg["Nombre fac"]=="Salud Pública":
        reg["Nombre fac"]="Facultad de Salud Pública"
    if reg["Nombre fac"]=="Agrarias":
        reg["Nombre fac"]="Facultad de Ciecias Agrarias"
    if reg["Nombre fac"]=="Bibliotecología":
        reg["Nombre fac"]="Escuela Interamericana de Bibliotecología"
    if reg["Nombre fac"]=="Enfermería":
        reg["Nombre fac"]="Facultad de Enfermería"
    if reg["Nombre fac"]=="Educación Física":
        reg["Nombre fac"]="Instituto Universitario de Educación Física y Deporte"
    if reg["Nombre fac"]=="Nutrición":
        reg["Nombre fac"]="Escuela de Nutrición y Dietética"
    if reg["Nombre fac"]=="Corp Ambiental":
        reg["Nombre fac"]="Corporación Ambiental"
    if reg["Nombre fac"]=="Est. Regionales":
        reg["Nombre fac"]="Instituto de Estudios Regionales"
        
    if not reg["Nombre fac"] in facs_inserted.keys():     
        entry=empty_affiliations()
        entry["updated"].append({"time":int(time()),"source":"staff"})
        entry["names"].append({"name":reg["Nombre fac"],"lang":"es"})
        entry["types"].append({"source":"staff","type":"faculty"})
        entry["relations"].append({"id":udea_reg["_id"],"names":udea_reg["names"]})
        
        response=colombia["affiliations"].insert_one(entry)
        facs_inserted[reg["Nombre fac"]]=response.inserted_id
    cedula_fac[reg["cedula"]]=reg["Nombre fac"]
    if not reg["Nombre cencos"] in deps_inserted.keys():
        entry=empty_affiliations()
        entry["updated"].append({"time":int(time()),"source":"staff"})
        entry["names"].append({"name":reg["Nombre cencos"],"lang":"es"})
        entry["types"].append({"source":"staff","type":"department"})
        entry["relations"].append({"id":udea_reg["_id"],"names":udea_reg["names"]})
        
        response=colombia["affiliations"].insert_one(entry)
        deps_inserted[reg["Nombre cencos"]]=response.inserted_id
    cedula_dep[reg["cedula"]]=reg["Nombre cencos"]
    if not (reg["Nombre fac"],reg["Nombre cencos"]) in fac_dep_pair:
        fac_dep_pair.append((reg["Nombre fac"],reg["Nombre cencos"]))

In [178]:
#updating relations between faculties and departments
for fac,dep in fac_dep_pair:
    fac_id=facs_inserted[fac]
    dep_id=deps_inserted[dep]
    dep_reg=colombia["affiliations"].find_one({"_id":ObjectId(dep_id)})
    fac_reg=colombia["affiliations"].find_one({"_id":ObjectId(fac_id)})
    colombia["affiliations"].update_one({"_id":fac_reg["_id"]},
                                        {"$push":{
                                            "relations":{
                                                "id":dep_reg["_id"],
                                                "names":dep_reg["names"],"types":dep_reg["types"]}}})
    colombia["affiliations"].update_one({"_id":dep_reg["_id"]},
                                        {"$push":{
                                            "relations":{
                                                "id":fac_reg["_id"],
                                                "names":fac_reg["names"],"types":fac_reg["types"]}}})

In [44]:
len(cedula_fac.keys())

1979

In [14]:
scienti=client["scienti_111"]["products"] 

### updating institutions from scienti

In [179]:
inst_updated=[]

In [101]:
fuzz.ratio("UNIVERSIDAD  INDUSTRIAL DE SANTANDER".lower(),"Universidad Industrial de Santander".lower())

99

In [ ]:
count=0
for cod_inst in scienti.distinct("group.institution.TXT_NIT"):
    if cod_inst in inst_updated:
        continue
    #inst=inst["group"][0]["institution"][0]
    reg_scienti=scienti.find_one({"group.institution.TXT_NIT":cod_inst})
    for inst in reg_scienti["group"][0]["institution"]:
        if inst["TXT_NIT"] in inst_updated:
            continue
        token=inst["NME_INST"]
        stopwords=["y","and","de","la","los","las","el","o","or","un","una","uno","en","por","para","según","a","ante",
          "con","de","sin","so","tras","e","u","del","and","or","from","to","after","about","by","in","out","next",
          "under","our","your","yours","them","their","my","it","we","have","had","be","do","are","him","her","hers","his",
          "then","where","why","how","what","which","who","whom","all","any","both","each","few","at","this","these","those",
          "that","if","as","with","while","against","about","here","there","off","of","-"]
        inst_name=" ".join([w for w in token.lower().split() if w not in stopwords])
        inst_name=inst_name.replace("universidad","").replace("institución universitaria","").replace("industrial","")
        inst_name=inst_name.replace("corporación","").replace("fundación","").replace("instituto","").strip()
        col_list=colombia["affiliations"].find({"$text":{"$search":inst_name}}).limit(30)
        reg_col=None
        name=None
        highest_score=0
        highest_name=None
        if "colciencias" in inst_name:
            reg_col=colombia["affiliations"].find_one({"_id" : ObjectId("637feecb71459ce0bcb7ec6c")})
            name=reg_col["names"][0]["name"]
        if inst["NME_INST"]=="UNIVERSIDAD CATOLICA DE ORIENTE":
            reg_col=colombia["affiliations"].find_one({"_id" : ObjectId("637feecf71459ce0bcb80944")})
            name=reg_col["names"][0]["name"]
        if not reg_col:
            for reg in col_list:
                for name in reg["names"]:
                    if inst["NME_INST"].lower()==name["name"].lower():
                        name=name["name"]
                        reg_col=reg
                        break
                if reg_col:
                    break
                for name in reg["names"]:
                    score=fuzz.ratio(inst["NME_INST"].lower(),name["name"].lower())
                    if score>90:
                        name=name["name"]
                        reg_col=reg
                        break
                    elif score>70:
                        score=fuzz.partial_ratio(inst["NME_INST"].lower(),name["name"].lower())
                        if score>93:
                            reg_col=reg
                            name=name["name"]
                            break
                        else:
                            if score>highest_score:
                                highest_score=score
                                highest_name=name["name"]
                    else:
                        if score> highest_score:
                            highest_score=score
                            highest_name=name["name"]
                if reg_col:
                    break
        if reg_col:
            reg_col["updated"].append({"source":"scienti","time":int(time())})
            reg_col["external_ids"].append({"source":"minciencias","id":inst["COD_INST"]})
            reg_col["external_ids"].append({"source":"nit","id":inst["TXT_NIT"]+"-"+inst["TXT_DIGITO_VERIFICADOR"]})
            if not inst["SGL_INST"] in reg_col["abbreviations"]:
                reg_col["abbreviations"].append(inst["SGL_INST"])
            if "URL_HOME_PAGE" in inst.keys():
                if not {"source":"site","url":inst["URL_HOME_PAGE"]} in reg_col["external_urls"]:
                    reg_col["external_urls"].append({"source":"site","url":inst["URL_HOME_PAGE"]})
            #print(reg_col)
            print("Updated: ",inst["NME_INST"]," - ",name)
            colombia["affiliations"].update_one({"_id":reg_col["_id"]},
                                               {"$set":{
                                                   "updated":reg_col["updated"],
                                                   "external_ids":reg_col["external_ids"],
                                                   "abbreviations":reg_col["abbreviations"],
                                                   "external_urls":reg_col["external_urls"]
                                               }})
            inst_updated.append(inst["TXT_NIT"])
        else:
            print(inst_name)
            print("Almost similar (",highest_score,"): ",inst["NME_INST"]," - ",highest_name)
    count+=1
    #print(inst)
print(count,len(inst_updated))

### Inserting groups from scienti

In [138]:
def extract_subject(subjects,data):
    subjects.append({
        "id":"",
        "name":data["TXT_NME_AREA"],
        "level":data["NRO_NIVEL"],
        "external_ids":[{"source":"OCDE","id":data["COD_AREA_CONOCIMIENTO"]}]
    })
    if "knowledge_area" in data.keys():
        extract_subject(subjects,data["knowledge_area"][0])
    return subjects


In [180]:
count=0
for group_id in scienti.distinct("group.COD_ID_GRUPO"):
    db_reg=colombia["affiliations"].find_one({"external_ids.id":group_id})
    if db_reg:
        continue
    entry=empty_affiliations()
    entry["updated"].append({"time":int(time()),"source":"scienti"})
    entry["external_ids"].append({"source":"minciencias","id":group_id})
    entry["types"].append({"source":"scienti","type":"group"})
    
    group=scienti.find_one({"group.COD_ID_GRUPO":group_id})
    group=group["group"][0]
    #print(group.keys())
    if group:
        entry["names"].append({"name":group["NME_GRUPO"],"lang":"es"})
        entry["birthdate"]=int(dt.datetime.strptime(str(group["ANO_FORMACAO"])+"-"+str(group["MES_FORMACAO"]),"%Y-%m").timestamp())
        if group["STA_ELIMINADO"]=="F":
            entry["status"].append({"source":"minciencias","status":"activo"})
        if group["STA_ELIMINADO"]=="T" or group["STA_ELIMINADO"]=="V":
            entry["status"].append({"source":"minciencias","status":"eliminado"})
        
        entry["relations"].append({"id":udea_reg["_id"],"names":udea_reg["names"]})
        
        entry["description"].append({
            "source":"scienti",
            "description":{
                "TXT_PLAN_TRABAJO":group["TXT_PLAN_TRABAJO"] if "TXT_PLAN_TRABAJO" in group.keys() else "",
                "TXT_ESTADO_ARTE":group["TXT_ESTADO_ARTE"] if "TXT_ESTADO_ARTE" in group.keys() else "",
                "TXT_OBJETIVOS":group["TXT_OBJETIVOS"]if "TXT_OBJETIVOS" in group.keys() else "",
                "TXT_PROD_DESTACADA":group["TXT_PROD_DESTACADA"]if "TXT_PROD_DESTACADA" in group.keys() else "",
                "TXT_RETOS":group["TXT_RETOS"]if "TXT_RETOS" in group.keys() else "",
                "TXT_VISION":group["TXT_VISION"] if "TXT_VISION" in group.keys() else ""
            }
        })
        
        if "TXT_CLASIF" in group.keys() and "DTA_CLASIF" in group.keys():
            entry["ranking"].append({
                "source":"scienti",
                "rank":group["TXT_CLASIF"],
                "from_date":int(dt.datetime.strptime(group["DTA_CLASIF"].split(", ")[-1].replace(" GMT",""),"%d %b %Y %H:%M:%S").timestamp()),
                "to_date":int(dt.datetime.strptime(group["DTA_FIN_CLASIF"].split(", ")[-1].replace(" GMT",""),"%d %b %Y %H:%M:%S").timestamp())
            })
        subjects=extract_subject([],group["knowledge_area"][0])
        if len(subjects)>0:
            entry["subjects"].append({
                "source":"OCDE",
                "subjects":subjects
            }) 
        
    count+=1
    #print(group["knowledge_area"])
    #print("----------------------")
    #print(entry["subjects"])
    #break
    colombia["affiliations"].insert_one(entry)
print(count)

402


#### Updating group relations with the institutions they "belong to"

In [181]:
for group in colombia["affiliations"].find({"types.type":"group"}):
    relations=[]
    cod=None
    for ext in group["external_ids"]:
        if ext["source"]=="minciencias":
            cod=ext["id"]
    if cod:
        for reg in scienti.find({"group.COD_ID_GRUPO":cod}):
            for inst in reg["group"][0]["institution"]:
                db_inst=colombia["affiliations"].find_one({"external_ids.id":inst["TXT_NIT"]+"-"+inst["TXT_DIGITO_VERIFICADOR"]})
                if db_inst:
                    rel_entry={"names":db_inst["names"],"id":db_inst["_id"],"types":db_inst["types"]}
                    if not rel_entry in relations:
                        relations.append(rel_entry)
        colombia["affiliations"].update_one({"_id":group["_id"]},
                                            {"$push":{
                                                "relations":{
                                                    "$each":relations
                                                }
                                            }}
                                           )

#### Inserting groups in relations from their institutions

In [182]:
for group in colombia["affiliations"].find({"types.type":"group"}):
    cod=None
    for ext in group["external_ids"]:
        if ext["source"]=="minciencias":
            cod=ext["id"]
    if cod:
        for reg in scienti.find({"group.COD_ID_GRUPO":cod}):
            for inst in reg["group"][0]["institution"]:
                db_inst=colombia["affiliations"].find_one({"external_ids.id":inst["TXT_NIT"]+"-"+inst["TXT_DIGITO_VERIFICADOR"]})
                if db_inst:
                    #print(db_inst["relations"])
                    rel_entry={"names":group["names"],"id":group["_id"],"types":group["types"]}
                    if not rel_entry in db_inst["relations"]:
                        colombia["affiliations"].update_one({"_id":db_inst["_id"]},{"$push":{"relations":rel_entry}})
                        #db_inst["relations"].append(rel_entry)
                        #print(db_inst["relations"])

In [243]:
#report a sample of two groups, two faculties, two departments, two institutions
sample=[]
for inst in colombia["affiliations"].find({"names.name":{"$in":["Universidad de Antioquia","Universidad de Caldas"]}}):
    sample.append(inst)
for fac in colombia["affiliations"].find({"names.name":{"$in":["Facultad de Ciencias Exactas y Naturales","Facultad de Ciencias Sociales y Humanas"]}}):
    sample.append(fac)
for dep in colombia["affiliations"].find({"names.name":{"$in":["Instituto de Física","Departamento de Sociología"]}}):
    sample.append(dep)
for group in colombia["affiliations"].find({"names.name":{"$in":["Grupo de Fenomenologia de Interacciones Fundamentales","Redes y Actores Sociales"]}}):
    sample.append(group)
with open("/current/data/colombia/sample_udea/sample_affiliations.json", "w") as outfile:
    json.dump(sample, outfile,cls=JSONEncoder)

### Inserting authors from "Talento Humano UdeA"

In [15]:
def empty_person():
    entry = {
        "updated":[],
        "full_name":"",
        "first_names":[],
        "last_names":"",
        "initials":"",
        "aliases":[],
        "affiliations":[],
        "keywords":[],
        "external_ids": [],
        "sex":"",
        "ranking":[],
        "birthplace":{},
        "birthdate":-1,
        "degrees":[],
        "subjects":[]
    }
    return entry

In [ ]:
th.iloc[0]

In [247]:
for idx in list(cedula_dep.keys()):
    check_db=colombia["person"].find_one({"external_ids.id":idx})
    if check_db:
        continue
    #if idx != "98554575":
    #    continue
    entry=empty_person()
    entry["updated"].append({"time":int(time()),"source":"staff"})

    for i,reg in th[th["cedula"]==idx].iterrows():
        aff_time=int(dt.datetime.strptime(reg["fecha_vin"],"%Y-%m-%d %H:%M:%S").timestamp())
        udea_aff={"id":udea_reg["_id"],"names":udea_reg["names"],"types":udea_reg["types"],"start_date":aff_time,"end_date":-1}
        if not udea_aff in entry["affiliations"]:
            entry["affiliations"].append(udea_aff)
        if reg["tipo_doc"]=="CC":
            id_entry={"source":"Cédula de Ciudadanía","id":idx}
            if not id_entry in entry["external_ids"]:
                entry["external_ids"].append(id_entry)
        elif reg["tipo_doc"]=="CE":
            id_entry={"source":"Cédula de Extranjería","id":idx}
            if not id_entry in entry["external_ids"]:
                entry["external_ids"].append(id_entry)
        if not reg["nombre"].lower() in entry["aliases"]:
            entry["aliases"].append(reg["nombre"].lower())
        dep=colombia["affiliations"].find_one({"names.name":reg["Nombre cencos"]})
        if dep:
            dep_affiliation={"id":dep["_id"],"names":dep["names"],"types":dep["types"],"start_date":aff_time,"end_date":-1}
            if not dep_affiliation in entry["affiliations"]:
                entry["affiliations"].append(dep_affiliation)
        fac=colombia["affiliations"].find_one({"names.name":reg["Nombre fac"]})
        if fac:
            fac_affiliation={"id":fac["_id"],"names":fac["names"],"types":fac["types"],"start_date":aff_time,"end_date":-1}
            if not fac_affiliation in entry["affiliations"]:
                entry["affiliations"].append(fac_affiliation)
        entry["birthdate"]=int(dt.datetime.strptime(reg["fecha_nac"],"%Y-%m-%d %H:%M:%S").timestamp())
        entry["sex"]=reg["sexo"].lower()
        degree={"date":"","degree":reg["nivelacad"],"id":"","institutions":[],"source":"staff"}
        if not degree in entry["degrees"]:
            entry["degrees"].append(degree)
        ranking={"date":"","rank":reg["categoria"],"source":"staff"}
        if not ranking in entry["ranking"]:
            entry["ranking"].append(ranking)
        
    #print(json.dumps(json.loads(json_util.dumps(entry)),indent=2))
    #print(entry)
    #break
    colombia["person"].insert_one(entry)

In [44]:
def fac_translation(reg):
    reg["Nombre fac"]=reg["Nombre fac"].strip()
    if reg["Nombre fac"]=='Vic Docencia':
        reg["Nombre fac"]="Vicerrectoría de Docencia"
    if reg["Nombre fac"]=="Exactas":
        reg["Nombre fac"]="Facultad de Ciencias Exactas y Naturales"
    if reg["Nombre fac"]=="Sociales":
        reg["Nombre fac"]="Facultad de Ciencias Sociales y Humanas"
    if reg["Nombre fac"]=="Derecho":
        reg["Nombre fac"]="Facultad de Derecho y Ciencias Políticas"
    if reg["Nombre fac"]=="Agrarias":
        reg["Nombre fac"]="Facultad de Ciencias Agrarias"
    if reg["Nombre fac"]=="Est. Políticos":
        reg["Nombre fac"]="Institutio de Estudios Políticos"
    if reg["Nombre fac"]=="Artes":
        reg["Nombre fac"]="Facultad de Artes"
    if reg["Nombre fac"]=="Odontología":
        reg["Nombre fac"]="Facultad de Odontología"
    if reg["Nombre fac"]=="Comunicaciones":
        reg["Nombre fac"]="Facultad de Comunicaciones y Filología"
    if reg["Nombre fac"]=="Educación":
        reg["Nombre fac"]="Facultad de Educación"
    if reg["Nombre fac"]=="Idiomas":
        reg["Nombre fac"]="Escuela de Idiomas"
    if reg["Nombre fac"]=="Filosofía":
        reg["Nombre fac"]="Instituto de Filosofía"
    if reg["Nombre fac"]=="Económicas":
        reg["Nombre fac"]="Facultad de Ciencias Económicas"
    if reg["Nombre fac"]=="Ingeniería":
        reg["Nombre fac"]="Facultad de Ingeniería"
    if reg["Nombre fac"]=="Medicina":
        reg["Nombre fac"]="Facultad de Medicina"
    if reg["Nombre fac"]=="Farmacéuticas":
        reg["Nombre fac"]="Facultad de Ciencias Farmacéuticas y Alimentarias"
    if reg["Nombre fac"]=="Microbiología":
        reg["Nombre fac"]="Escuela de Microbiología"
    if reg["Nombre fac"]=="Salud Pública":
        reg["Nombre fac"]="Facultad de Salud Pública"
    if reg["Nombre fac"]=="Agrarias":
        reg["Nombre fac"]="Facultad de Ciecias Agrarias"
    if reg["Nombre fac"]=="Bibliotecología":
        reg["Nombre fac"]="Escuela Interamericana de Bibliotecología"
    if reg["Nombre fac"]=="Enfermería":
        reg["Nombre fac"]="Facultad de Enfermería"
    if reg["Nombre fac"]=="Educación Física":
        reg["Nombre fac"]="Instituto Universitario de Educación Física y Deporte"
    if reg["Nombre fac"]=="Nutrición":
        reg["Nombre fac"]="Escuela de Nutrición y Dietética"
    if reg["Nombre fac"]=="Corp Ambiental":
        reg["Nombre fac"]="Corporación Ambiental"
    if reg["Nombre fac"]=="Est. Regionales":
        reg["Nombre fac"]="Instituto de Estudios Regionales"
    return reg["Nombre fac"]

In [69]:
#Fixing faculties for authors already inserted from talento humano and wothout faculties
for idx in th["cedula"].unique():
    #print(idx)
    author=colombia["person"].find_one({"external_ids.id":idx})
    affs=[]
    for aff in author["affiliations"]:
        if aff["types"][0]["type"]!="faculty":
            affs.append(aff)

    reg=th[th["cedula"]==idx].iloc[0]
    fac_name=str(reg["Nombre fac"])
    fac_translated=fac_translation(reg)
    aff_time=int(dt.datetime.strptime(reg["fecha_vin"],"%Y-%m-%d %H:%M:%S").timestamp())
    
    fac_db=colombia["affiliations"].find_one({"types.type":"faculty","names.name":fac_translated})
    if fac_db:
        affs.append(
            {
                "id":fac_db["_id"],
                "names":fac_db["names"],
                "types":fac_db["types"],
                "start_date":aff_time,"end_date":-1
            }
        )
        colombia["person"].update_one({"_id":author["_id"]},{"$set":{"affiliations":affs}})
    else:
        print(idx)

### Inserting/updating authors from scienti db

In [248]:
updated=[]

In [249]:
#updating authors already inserted with the information in scienti dump
found=0
missing=0
for person in colombia["person"].find():
    if person["_id"] in updated:
        continue
    idx=None
    for ext in person["external_ids"]:
        if ext["source"]=="Cédula de Ciudadanía":
            idx=ext["id"]
        elif ext["source"]=="Cédula de Extranjería":
            idx=ext["id"]
    scienti_reg=scienti.find_one({"author.NRO_DOCUMENTO_IDENT":idx})
    if scienti_reg:
        author=scienti_reg["author"][0]
        found+=1
        person["external_ids"].append({"source":"scienti","id":author["COD_RH"]})
        if "COD_ORCID" in author.keys():
            if author["COD_ORCID"]:
                person["external_ids"].append({"source":"orcid","id":author["COD_ORCID"]})
        person["first_names"]=author["TXT_NAMES_RH"]
        person["last_names"]=[]
        if "TXT_PRIM_APELL" in author.keys():
            person["last_names"].append(author["TXT_PRIM_APELL"])
        if "TXT_SEG_APELL" in author.keys():
            person["last_names"].append(author["TXT_SEG_APELL"])
        person["last_names"]=" ".join(person["last_names"])
        person["updated"].append({"time":int(time()),"source":"scienti"})
        colombia["person"].update_one({"_id":person["_id"]},{"$set":{
            "external_ids":person["external_ids"],
            "first_names":person["first_names"],
            "last_names":person["last_names"],
            "full_name":person["first_names"]+" "+person["last_names"],
            "updated":person["updated"]
        }})
        updated.append(person["_id"])
    else:
        missing+=1
print(found,missing,colombia["person"].count_documents({}))

1263 716 1979


In [250]:
people_groups=0
for person in colombia["person"].find():
    idx=None
    updatable=False
    for ext in person["external_ids"]:
        if ext["source"]=="Cédula de Ciudadanía":
            idx=ext["id"]
        elif ext["source"]=="Cédula de Extranjería":
            idx=ext["id"]
    for prod in scienti.find({"author.NRO_DOCUMENTO_IDENT":idx}):
        group_entry={}
        group_db=colombia["affiliations"].find_one({"external_ids.id":prod["group"][0]["COD_ID_GRUPO"]})
        if group_db:
            aff_found=False
            for aff in person["affiliations"]:
                if group_db["_id"] == aff["id"]:
                    aff_found=True
                    break
            if aff_found:
                continue
            time_str=""
            if len(str(prod["NRO_ANO_PRESENTA"]))==4:
                time_str+=str(prod["NRO_ANO_PRESENTA"])
            else:
                continue
            if len(str(prod["NRO_MES_PRESENTA"]))<2:
                time_str+="-0"+str(prod["NRO_MES_PRESENTA"])
            elif len(str(prod["NRO_MES_PRESENTA"]))==2:
                time_str+="-"+str(prod["NRO_MES_PRESENTA"])
            aff_time=int(dt.datetime.strptime(time_str,"%Y-%m").timestamp())
            group_entry={"id":group_db["_id"],"names":group_db["names"],"types":group_db["types"],"start_date":aff_time,"end_date":-1}
            if not group_entry in person["affiliations"]:
                person["affiliations"].append(group_entry)
                updatable=True
    if updatable:
        colombia["person"].update_one({"_id":person["_id"]},{"$set":{"affiliations":person["affiliations"]}})

In [251]:
#fixing names for the authors inserted from staff but missing from scienti
for person in colombia["person"].find({"full_name":""}):
    #print(person)
    names=split_names(person["aliases"][0])
    colombia["person"].update_one({"_id":person["_id"]},{"$set":{
        "full_name":names["NOMBRE COMPLETO"],
        "first_names":names["NOMBRES"],
        "last_names":names["PRIMER APELLIDO"]+" "+names["SEGUNDO APELLIDO"]
    }})

In [70]:
empty_person()

{'updated': [],
 'full_name': '',
 'first_names': '',
 'last_names': '',
 'initials': '',
 'aliases': [],
 'affiliations': [],
 'keywords': [],
 'external_ids': [],
 'sex': '',
 'ranking': [],
 'birthplace': {},
 'birthdate': -1,
 'degrees': [],
 'subjects': []}

In [92]:
#inserting the authors found in scienti but not in the staff file
already=0
count=0
inserted=0
for rh in scienti.distinct("author.COD_RH"):
    autor_db=None
    count+=1
    #print(rh)
    author_db=colombia["person"].find_one({"external_ids.id":rh})
    if author_db:
        already+=1
        continue
    author_scienti=scienti.find_one({"author.COD_RH":rh})
    if author_scienti:
        author=author_scienti["author"][0]
        if "NRO_DOCUMENTO_IDENT" in author.keys():
            author_db=colombia["person"].find_one({"external_ids.id":author["NRO_DOCUMENTO_IDENT"]})
            if author_db:
                already+=1
        if not author_db:
            if "COD_ORCID" in author.keys():
                author_db=colombia["person"].find_one({"external_ids.id":author["COD_ORCID"]})
                if author_db:
                    already+=1
        if not author_db:
            if "AUTHOR_ID_SCP" in author.keys():
                author_db=colombia["person"].find_one({"external_ids.id":author["AUTHOR_ID_SCP"]})
                if author_db:
                    already+=1
        if not author_db:
            entry=empty_person()
            entry["updated"].append({"time":int(time()),"source":"scienti"})
            if author["TPO_DOCUMENTO_IDENT"]=="P":
                entry["external_ids"].append({"source":"Passport","id":author["NRO_DOCUMENTO_IDENT"]})
            if author["TPO_DOCUMENTO_IDENT"]=="C":
                entry["external_ids"].append({"source":"Cédula de Ciudadanía","id":author["NRO_DOCUMENTO_IDENT"]})
            if author["TPO_DOCUMENTO_IDENT"]=="E":
                entry["external_ids"].append({"source":"Cédula de Extranjería","id":author["NRO_DOCUMENTO_IDENT"]})
            entry["external_ids"].append({"source":"scienti","id":author["COD_RH"]})
            if "COD_ORCID" in author.keys():
                if author["COD_ORCID"]:
                    entry["external_ids"].append({"source":"orcid","id":author["COD_ORCID"]})
            entry["first_names"]=author["TXT_NAMES_RH"]
            entry["last_names"]=[]
            if "TXT_PRIM_APELL" in author.keys():
                entry["last_names"].append(author["TXT_PRIM_APELL"])
            if "TXT_SEG_APELL" in author.keys():
                entry["last_names"].append(author["TXT_SEG_APELL"])
            entry["last_names"]=" ".join(entry["last_names"])
            entry["full_name"]=entry["first_names"]+" "+entry["last_names"]
            if "TXT_CITACION_BIBLIO" in author.keys():
                entry["aliases"].append(author["TXT_CITACION_BIBLIO"].lower())
            if "TPO_SEXO" in author.keys():
                entry["sex"]=author["TPO_SEXO"].lower()
            if "TPO_PERFIL" in author.keys():
                ranking={"date":"","rank":author["TPO_PERFIL"],"source":"scienti"}
                if not ranking in entry["ranking"]:
                    entry["ranking"].append(ranking)
            if "institution" in author_scienti.keys():
                aff_db=colombia["affiliations"].find_one({"external_ids.id":author_scienti["institution"][0]["COD_INST"]})
                if aff_db:
                    entry["affiliations"].append({
                        "id":aff_db["_id"],
                        "names":aff_db["names"],
                        "types":aff_db["types"],
                        "start_date":-1,
                        "end_date":-1
                    })
            if "group" in author_scienti.keys():
                aff_db=colombia["affiliations"].find_one({"external_ids.id":author_scienti["group"][0]["COD_ID_GRUPO"]})
                if aff_db:
                    entry["affiliations"].append({
                        "id":aff_db["_id"],
                        "names":aff_db["names"],
                        "types":aff_db["types"],
                        "start_date":-1,
                        "end_date":-1
                    })
            
            colombia["person"].insert_one(entry)
            inserted+=1
print(already,inserted,count)

1265 2091 3356


In [94]:
#Check if there is repeated authors
repeated_ids=[]
with client.start_session() as session:
    old=dt.datetime.now()
    for author in colombia["person"].find():
        for ext in author["external_ids"]:
            count=colombia["person"].count_documents({"external_ids.id":ext["id"]})
            if count>1:
                repeated_ids.append(author["_id"])
        delta=dt.datetime.now()-old
        if delta.seconds>240:
            print(len(repeated_ids))
            client.admin.command('refreshSessions', [session.session_id], session=session)
            old=dt.datetime.now()

In [ ]:
#Filling initials variable
for person in colombia["person"].find({"initials":""}):
    initials="".join([p[0].upper() for p in person["first_names"].split()])
    colombia["person"].update_one({"_id":person["_id"]},{"$set":{"initials":initials}})

In [254]:
#report authors
sample=[]
for person in colombia["person"].find({"last_names":{"$in":["Restrepo Quintero","Velez Cuartas","Uribe Tirado"]}}):
    sample.append(person)
with open("/current/data/colombia/sample_udea/sample_person.json", "w") as outfile:
    json.dump(sample, outfile,cls=JSONEncoder)

In [ ]:
#changing lastnames from one string to a list
hard_scienti=[]
hard_others=[]
modified=0
skipped=0
total=0
for author in colombia["person"].find():
    total+=1
    if isinstance(author["last_names"],list):
        skipped+=1
        continue
    ln_split=author["last_names"].split()
    if len(ln_split)>2:
        updated_from=[]
        for up in author["updated"]:
            updated_from.append(up["source"])
        print(updated_from)
        print(author["first_names"],ln_split)
        if "scienti" in updated_from:
            hard_scienti.append(author["_id"])
        else:
            hard_others.append(author["_id"])
    else:
        #pass
        colombia["person"].update_one({"_id":author["_id"]},{"$set":{"last_names":ln_split}})
        modified+=1
print(skipped,modified,total)

In [122]:
split_names("DA SILVEIRA ARRUDA NATALIA")

{'NOMBRE COMPLETO': 'Natalia Da Silveira Arruda',
 'PRIMER APELLIDO': 'Da Silveira',
 'SEGUNDO APELLIDO': 'Arruda',
 'NOMBRES': 'Natalia',
 'INICIALES': 'N.'}

In [110]:
skipped=0
len(hard_scienti),len(hard_others)
for idx in hard_scienti:
    author=colombia["person"].find_one({"_id":idx})
    if isinstance(author["last_names"],list):
        skipped+=1
        continue
    extid=None
    for ext in author["external_ids"]:
        if ext["source"]=="scienti":
            extid=ext["id"]
            break
    author_scienti=scienti.find_one({"author.COD_RH":extid})
    author_scienti=author_scienti["author"][0]
    last_names=[]
    if "TXT_PRIM_APELL" in author_scienti.keys():
        last_names.append(author_scienti["TXT_PRIM_APELL"])
    if "TXT_SEG_APELL" in author_scienti.keys():
        last_names.append(author_scienti["TXT_SEG_APELL"])
    colombia["person"].update_one({"_id":author["_id"]},{"$set":{"last_names":last_names}})
print(skipped)

31


In [125]:
skipped=0
for idx in hard_others:
    author=colombia["person"].find_one({"_id":idx})
    if isinstance(author["last_names"],list):
        skipped+=1
        continue
    extid=None
    for ext in author["external_ids"]:
        if ext["source"]=="scienti":
            extid=ext["id"]
            break
    last_names=[]
    if author["full_name"]=="Manuel Eusebio Molina Del Aguila":
        last_names=["Molina", "Del Aguila"]
    if author["full_name"]=="Victoria Eugenia Gonzalez De Franco":
        last_names=["Gonzalez", "De Franco"]
    if author["full_name"]=="Natalia Da Silveira Arruda":
        last_names=["Da Silveira", "Arruda"]
    else:
        name_dict=split_names(author["full_name"])
        if name_dict["PRIMER APELLIDO"]:
            last_names.append(name_dict["PRIMER APELLIDO"])
        if name_dict["SEGUNDO APELLIDO"]:
            last_names.append(name_dict["SEGUNDO APELLIDO"])
    colombia["person"].update_one({"_id":author["_id"]},{"$set":{"last_names":last_names}})

In [126]:
#checking if there are last_names not on a list
total=0
skipped=0
wrong=0
for author in colombia["person"].find():
    total+=1
    if isinstance(author["last_names"],list):
        skipped+=1
        continue
    else:
        wrong+=1
print(total,skipped,wrong)

4070 4070 0


### Inserting authors from minciencias datos abiertos file

In [97]:
investigadores_minciencias=pd.read_csv("/current/data/colombia/scienti-abiertos/2-Investigadores_Reconocidos_por_convocatoria.csv",dtype={"ID_PERSONA_PR":str})

### Inserting authors from ranking (puntaje) file

In [74]:
puntaje = pd.read_excel("/current/data/colombia/udea/produccion 2018-2022 al 27 oct 2022.xlsx",
                        dtype={"cedula":str}
                       )

In [11]:
puntaje.columns

Index(['cedula', 'nombre', 'tipo mat', 'capdescrip', 'tipo concep', 'tipo mov',
       'ptos', 'numero autores', 'año realiz', 'fecha vig', 'fecha aplica',
       'fecha pres', 'codigo', 'titulo', 'codigo mat', 'nombre rev o premio',
       'issn', 'numero acta', 'pais', 'idioma', 'fecha registro', 'DOI',
       'URL'],
      dtype='object')

In [ ]:
puntaje.iloc[0]

In [87]:
missing=[]
in_scienti=[]
for cedula in puntaje["cedula"].unique():
    products=puntaje[puntaje["cedula"]==cedula]
    author_db=colombia["person"].find_one({"external_ids.id":cedula})
    if author_db:
        continue
    missing.append(cedula)
    scienti_db=scienti.find_one({"author.NRO_DOCUMENTO_IDENT":cedula})
    if scienti_db:
        in_scienti.append(cedula)
print(len(missing),len(cedula))

179 8


In [72]:
missing=0
found=0
for idx in scienti.distinct("institution.COD_INST"):
    reg_db=colombia["affiliations"].find_one({"external_ids.id":idx})
    if reg_db:
        found+=1
    else:
        missing+=1
print(found,missing,found+missing)

17 26 43


In [84]:
missing=0
found=0
fixable=0
for idx in scienti.distinct("institution.COD_INST"):
    reg_db=colombia["affiliations"].find_one({"external_ids.id":idx})
    if reg_db:
        found+=1
    else:
        scienti_db=scienti.find_one({"institution.COD_INST":idx})
        scienti_db=scienti_db["institution"][0]
        reg_db=colombia["affiliations"].find_one({"names.name":{"$regex":"/"+scienti_db["NME_INST"]+"/i"}})
        if reg_db:
            fixable+=1
        else:
            print(scienti_db["NME_INST"],scienti_db["COD_INST"])
            missing+=1
print(found,fixable,missing,found+fixable+missing)

INSTITUTO TECNOLÓGICO METROPOLITANO DE MEDELLÍN 000000000085
MATH DECISION SAS 000000010032
TECNOLOGICO DE ANTIOQUIA INSTITUCION UNIVERSITARIA 000000000261
POLITÉCNICO COLOMBIANO JAIME ISAZA CADAVID 633300000884
UNIVERSIDAD AUTONOMA DE MANIZALES 000000009276
Universidad De Manizales - Umanizales 008800000884
CORPORACION UNIVERSIDAD DE LA COSTA 625400000885
UNIVERSIDAD CES 628200000886
UNIVERSIDAD ICESI 014700000887
UNIVERSIDAD DE CORDOBA 002600000881
SERVICIO NACIONAL DE APRENDIZAJE SENA 402100000888
UNIVERSIDAD SANTO TOMÁS, SECCIONAL BUCARAMANGA 015900000889
UNIVERSIDAD AUTONOMA DE BUCARAMANGA 015400000880
UNIVERSIDAD DEL TOLIMA 016100000882
INSTITUTO NACIONAL DE CANCEROLOGIA ESE 208900000889
UNIVERSIDAD EIA 626800000880
FUNDACIÓN HOSPITALARIA SAN VICENTE DE PAÚL 000000000552
Universidad del Sinú - Elías Bechara Zainum 000000000013
Fundación Universitaria Autónoma De Las Américas 000000000268
FUNDACIÓN UNIVERSITARIA CATÓLICA DEL NORTE 034000000889
UNIVERSIDAD SANTO TOMAS 000000006045


In [82]:
colombia["affiliations"].find_one({"names.name":{"$regex":"/universidad de antioquia/i"}})